In [1]:
import pandas as pd
import numpy as np
from os.path import join

In [5]:
pd_train = pd.read_csv('train_data.csv')
pd_test = pd.read_csv('test_data.csv')

In [15]:
import librosa

def extract_feature(file_name):
    audio_signal, sample_rate = librosa.load(file_name, sr=22050)
    spectrogram = librosa.stft(audio_signal, n_fft=512)
    spectrogram = np.abs(spectrogram)
    spectrogram_feature = np.mean(spectrogram.T, axis=0)

    power_spectrogram = spectrogram ** 2
    mel = librosa.feature.melspectrogram(S=power_spectrogram, sr=sample_rate)
    mel = librosa.power_to_db(mel)
    mel_spectrogram_feature = np.mean(mel.T, axis=0)
    
    mfccs = librosa.feature.mfcc(S=mel, n_mfcc=20)
    mfcc_feature = np.mean(mfccs.T, axis=0)

    return spectrogram_feature, mel_spectrogram_feature, mfcc_feature 

# **데이터 불러오기**

### csv파일에 저장된 파일 이름과 학습용 label을 로드하여 학습 및 평가용 데이터를 불러오세요.

In [17]:
from tqdm.notebook import tqdm
def load_data(data_info, isTrain=True):
    
    if isTrain:
        train_data = {'spectrogram':[],'mel':[],'mfcc':[]} #음성 feature들을 담는 dictionary
        train_label = [] #학습에 사용할 label을 담는 list
        
        file_list = data_info['file_name']
        emotion_list = data_info['emotion']
        for file_name, emotion in tqdm(zip(file_list, emotion_list)):

            spectrogram, mel, mfccs = extract_feature('train_data/train_data/' + file_name)
            train_data['spectrogram'].append(spectrogram)
            train_data['mel'].append(mel)
            train_data['mfcc'].append(mfccs)
            train_label.append(emotion)

        return train_data, np.array(train_label)
    
    else:
        test_data = {'spectrogram':[],'mel':[],'mfcc':[]} #음성 feature들을 담는 dictionary
        file_list = data_info['file_name']
    
        for file_name in tqdm(file_list):
            spectrogram, mel, mfccs = extract_feature('test_data/test_data/' + file_name)
            test_data['spectrogram'].append(spectrogram)
            test_data['mel'].append(mel)
            test_data['mfcc'].append(mfccs)

        return test_data

#DataFlair - Split the dataset
train_data, y_train = load_data(pd_train)
test_data = load_data(pd_test, isTrain=False)

0it [00:00, ?it/s]

  0%|          | 0/432 [00:00<?, ?it/s]

# 모델 학습 및 추론
위에서 우리는 
1. 음성 신호에 대하여 Short Time Fourier Transform(stft)를 통해 **spectrogram**을 구하고,
2. 구한 spectrogram에 mel-filter를 씌워 **mel-spectrogram**을 구했으며,
3. 마지막으로 mel-spectrogram에 Discrete cosine transform(DCT) 취해줌으로써 **mfcc**까지 계산하였습니다.

위에서 구한 feature들은 모두 음성 데이터를 활용하는 다양한 작업(사람 음성 분류, 음성을 통한 감정 분류 등)에서 모델 학습에 사용할 수 있는 feature들입니다.
각각의 **feature들에 따른 모델의 정확도가 얼마나 차이**나는지를 확인해봅시다.

- 베이스라인의 분류기는 **RandomForestClassifier**를 사용합니다.
- **random_state에 따른 성능 차이가 발생하오니 반드시 random_state를 1로 고정해주시길 바랍니다.**

In [18]:
#RandomForestClassifier로 음성 감정 분류 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
sample = pd.read_csv('sample_submit.csv')

for feature_name in train_data.keys():
    x_train = np.array(train_data[feature_name])
    x_test = np.array(test_data[feature_name])
    
    model = RandomForestClassifier(random_state=1)
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    sample['emotion'] = predict.reshape(-1,1)
    sample.to_csv(join(feature_name+'.csv'), index=False, header=True)
